In [ ]:
import mitdeeplearning as mdl

In [ ]:
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras import layers as L
import tensorflow.keras.preprocessing.image as imgPrep
import os
import pandas as pd
from tensorflow.keras.utils import plot_model

###Constant###
EPOCHS = 25
SMOOTHING_FAC = 0

In [ ]:
data_path = r'C:\Fairfax SEF\short_data.h5'
cache = h5py.File(data_path, 'r')
images = cache['images'][:]

In [ ]:
plt.imshow(images[0])

In [ ]:
labels = cache['labels'][:].astype(np.float32)
labels = labels.reshape(-1, 1)

In [ ]:
number_of_training_examples = images.shape[0]
image_dims = images.shape
n_train_samples = image_dims[0]
train_inds = np.random.permutation(np.arange(n_train_samples))
pos_train_inds = train_inds[labels[train_inds, 0] == 1.0 ]
neg_train_inds = train_inds[labels[train_inds, 0] != 1.0 ]

In [ ]:
def get_batch(n, only_faces=False, p_pos=None, p_neg=None, return_inds=False):
    if only_faces:
            selected_inds = np.random.choice(pos_train_inds, size=n, replace=False, p=p_pos)
    else:
        selected_pos_inds = np.random.choice(pos_train_inds, size=n//2, replace=False, p=p_pos)
        selected_neg_inds = np.random.choice(neg_train_inds, size=n//2, replace=False, p=p_neg)
        selected_inds = np.concatenate((selected_pos_inds, selected_neg_inds))
    sorted_inds = np.sort(selected_inds)
    train_img = (images[sorted_inds,:,:,::]).astype(np.float32)
    train_label = labels[sorted_inds,...]
    return (train_img, train_label, sorted_inds) if return_inds else (train_img, train_label)

In [ ]:
n_filters=12
def make_standard_classifier(n_outputs=1):
    model = tf.keras.Sequential()
    model.add(L.Conv2D(filters=1*n_filters, kernel_size=5, 
                       strides=2, padding='same', activation='relu'))
    model.add(L.BatchNormalization())

    model.add(L.Conv2D(filters=2*n_filters, kernel_size=5, 
                       strides=2, padding='same', activation='relu'))
    model.add(L.BatchNormalization())

    model.add(L.Conv2D(filters=4*n_filters, kernel_size=5, 
                       strides=2, padding='same', activation='relu'))
    model.add(L.BatchNormalization())

    model.add(L.Conv2D(filters=6*n_filters, kernel_size=5, 
                       strides=2, padding='same', activation='relu'))
    model.add(L.BatchNormalization())

    model.add(L.Conv2D(filters=8*n_filters, kernel_size=5, 
                       strides=2, padding='same', activation='relu'))
    model.add(L.BatchNormalization())

    model.add(L.Flatten())
    model.add(L.Dense(512))
    model.add(L.Dense(n_outputs, activation=None))
    return model
model = make_standard_classifier()

In [ ]:
batch_size = 32
num_epochs = EPOCHS 
learning_rate = 5e-4

optimizer = tf.keras.optimizers.Adam(learning_rate) 
loss_history = mdl.util.LossHistory(smoothing_factor=0.99) 
plotter = mdl.util.PeriodicPlotter(sec=2, scale='semilogy')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() 

    
@tf.function
def standard_train_step(x, y):
    with tf.GradientTape() as tape:
        # feed the images into the model
        logits = model(x) 
        
        # Compute the loss
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
        print(loss)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# The training loop!
for epoch in range(num_epochs):
    for idx in tqdm(range(n_train_samples//batch_size)):
        # Grab a batch of training data and propagate through the network
        x, y = get_batch(batch_size)
        loss = standard_train_step(x, y)

        # Record the loss and plot the evolution of the loss as a function of training
        loss_history.append(loss.numpy().mean())
        plotter.plot(loss_history.get())
        break
    break

In [ ]:
(batch_x, batch_y) = get_batch(1000)
y_pred_standard = tf.round(tf.nn.sigmoid(model.predict(batch_x)))
acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, y_pred_standard), tf.float32))

print("Standard CNN accuracy on (potentially biased) training set: {:.4f}".format(acc_standard.numpy()))

In [ ]:
def vae_loss_function(x, x_recon, mu, logsigma, kl_weight=0.0005):

    latent_loss = tf.reduce_sum(tf.exp(logsigma)+mu**2-1-logsigma, axis=1)*0.5


    reconstruction_loss = tf.reduce_mean(tf.abs(x-x_recon), axis=(1, 2, 3))

    vae_loss = kl_weight*latent_loss+reconstruction_loss

    return vae_loss

In [ ]:
def sampling(z_mean, z_logsigma):
    batch, latent_dim = z_mean.shape
    epsilon = tf.random.normal(shape=(batch, latent_dim))

    z = z_mean+tf.exp(0.5*z_logsigma)*epsilon
    return z

In [ ]:
def debiasing_loss_function(x, x_pred, y, y_logit, mu, logsigma):

    vae_loss = vae_loss_function(x, x_pred, mu, logsigma)


    classification_loss = tf.nn.sigmoid_cross_entropy_with_logits(y, y_logit)


    mel_indicator = tf.cast(tf.equal(y, 1), tf.float32)


    total_loss = classification_loss+mel_indicator*vae_loss

    return total_loss, classification_loss

In [ ]:
n_filters = 12 # base number of convolutional filters, same as standard CNN
latent_dim = 100 # number of latent variables

def make_face_decoder_network():
    # Functionally define the different layer types we will use

    # Build the decoder network using the Sequential API
    decoder = tf.keras.Sequential([
    # Transform to pre-convolutional generation
    L.Dense(units=8*8*8*n_filters, activation='relu'),  # 4x4 feature maps (with 6N occurances)
    L.Reshape(target_shape=(8, 8, 8*n_filters)),

    # Upscaling convolutions (inverse of encoder)
    L.Conv2DTranspose(filters=6*n_filters, kernel_size=5,  strides=2, padding='same', activation='relu'),
    L.Conv2DTranspose(filters=4*n_filters, kernel_size=5,  strides=2, padding='same', activation='relu'),
    L.Conv2DTranspose(filters=2*n_filters, kernel_size=5,  strides=2, padding='same', activation='relu'),
    L.Conv2DTranspose(filters=1*n_filters, kernel_size=5,  strides=2, padding='same', activation='relu'),
    L.Conv2DTranspose(filters=3, kernel_size=5,  strides=2, padding='same', activation='relu'),
    ])

    return decoder

In [ ]:
class LatentNet(tf.keras.Model):
  def __init__(self, latent_dim):
    super(LatentNet, self).__init__()
    self.latent_dim = latent_dim

    # Define the number of outputs for the encoder. Recall that we have 
    # `latent_dim` latent variables, as well as a supervised output for the 
    # classification.
    num_encoder_dims = 2*self.latent_dim + 1

    self.encoder = make_standard_classifier(num_encoder_dims)
    self.decoder = make_face_decoder_network()

  # function to feed images into encoder, encode the latent space, and output
  #   classification probability 
  def encode(self, x):
    # encoder output
    encoder_output = self.encoder(x)

    # classification prediction
    y_logit = tf.expand_dims(encoder_output[:, 0], -1)
    # latent variable distribution parameters
    z_mean = encoder_output[:, 1:self.latent_dim+1] 
    z_logsigma = encoder_output[:, self.latent_dim+1:]

    return y_logit, z_mean, z_logsigma

  # VAE reparameterization: given a mean and logsigma, sample latent variables
  def reparameterize(self, z_mean, z_logsigma):
    z = sampling(z_mean, z_logsigma)
    return z

  # Decode the latent space and output reconstruction
  def decode(self, z):
    reconstruction = self.decoder(z)
    return reconstruction

  def call(self, x): 
    y_logit, z_mean, z_logsigma = self.encode(x)

    z = self.reparameterize(z_mean, z_logsigma)

    recon = self.decode(z)
    return y_logit, z_mean, z_logsigma, recon

  def predict(self, x):
    y_logit, z_mean, z_logsigma = self.encode(x)
    return y_logit



In [ ]:
def get_latent_mu(images, latent_net, batch_size=1024):
  N = images.shape[0]
  mu = np.zeros((N, latent_dim))
  for start_ind in range(0, N, batch_size):
    end_ind = min(start_ind+batch_size, N+1)
    batch = (images[start_ind:end_ind]).astype(np.float32)/255.
    _, batch_mu, _ = latent_net.encode(batch)
    mu[start_ind:end_ind] = batch_mu
  return mu

In [ ]:
'''Function that recomputes the sampling probabilities for images within a batch
      based on how they distribute across the training data'''
def get_training_sample_probabilities(images, latent_net, bins=10, smoothing_fac=SMOOTHING_FAC): 
    print("Recomputing the sampling probabilities")
    
    mu = get_latent_mu(images, latent_net) # TODO

    # sampling probabilities for the images
    training_sample_p = np.zeros(mu.shape[0])
    
    for i in range(latent_dim):
      
        latent_distribution = mu[:,i]
        hist_density, bin_edges =  np.histogram(latent_distribution, density=True, bins=bins)

        bin_edges[0] = -float('inf')
        bin_edges[-1] = float('inf')
        

        bin_idx = np.digitize(latent_distribution, bin_edges) 
        hist_smoothed_density = hist_density + smoothing_fac
        hist_smoothed_density = hist_smoothed_density / np.sum(hist_smoothed_density)

        p = 1.0/(hist_smoothed_density[bin_idx-1])
        p = p/np.sum(p)
        training_sample_p = np.maximum(p, training_sample_p)
        break
        
    # final normalization
    training_sample_p /= np.sum(training_sample_p)

    return training_sample_p

In [ ]:
import IPython

In [ ]:
### Training the LatentNet ###

# Hyperparameters
batch_size = 32
learning_rate = 5e-4
latent_dim = 100


num_epochs = EPOCHS 

latent_net = LatentNet(latent_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate)


@tf.function
def debiasing_train_step(x, y):

  with tf.GradientTape() as tape:
    y_logit, z_mean, z_logsigma, x_recon = latent_net(x)

    loss, class_loss = debiasing_loss_function(x, x_recon, y, y_logit, z_mean, z_logsigma)

  grads = tape.gradient(loss, latent_net.trainable_variables)
  optimizer.apply_gradients(zip(grads, latent_net.trainable_variables))
  return loss

# get training faces from data loader
all_mel = images[pos_train_inds]

if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

# The training loop -- outer loop iterates over the number of epochs
for i in range(num_epochs):

  IPython.display.clear_output(wait=True)
  print("Starting epoch {}/{}".format(i+1, num_epochs))

  # Recompute data sampling proabilities
  '''TODO: recompute the sampling probabilities for debiasing'''
  p_mel = get_training_sample_probabilities(all_mel, latent_net) 
  
  # get a batch of training data and compute the training step
  for j in tqdm(range(n_train_samples // batch_size)):
    # load a batch of data
    (x, y) = get_batch(batch_size, p_pos=p_mel)
    # loss optimization
    loss = debiasing_train_step(x, y)
    
    # plot the progress every 200 steps
    if j % 500 == 0: 
      mdl.util.plot_sample(x, y, latent_net)

In [ ]:
latent_net_logits = [latent_net.predict(np.array(x.reshape(1, 256, 256, 3), dtype=np.float32)) for x in batch_x]
latent_net_probs = tf.squeeze(tf.sigmoid(latent_net_logits))
preds = [np.array([1.0]) if pred > 0.5 else np.array([0.0]) for pred in latent_net_probs]
preds = np.array(preds)
acc_standard = tf.reduce_mean(tf.cast(tf.equal(batch_y, preds), tf.float32))

print("LatentNet accuracy on training set: {:.4f}".format(acc_standard.numpy()))

In [ ]:
def test(path):
    latent_net_preds = []
    reg_preds = []
    for img in os.listdir(path):
        full_path = os.path.join(path, img)
        loaded = imgPrep.load_img(full_path, target_size=(256, 256))
        arr = imgPrep.img_to_array(loaded)/255.0
        pred1 = tf.sigmoid(latent_net.predict(arr.reshape(1, 256, 256, 3)))
        pred2 = tf.sigmoid(model.predict(arr.reshape(1, 256, 256, 3)))
        latent_net_preds.append(np.array(pred1)[0][0]); reg_preds.append(np.array(pred2)[0][0])
    print(f'LatentNet Confidence: {sum(latent_net_preds)/len(latent_net_preds)}')
    print(f'Standard Confidence: {sum(reg_preds)/len(reg_preds)}')
    return latent_net_preds, reg_preds

In [ ]:
version = '5'
normal_hair = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Hair Density/Normal'
mel_hair = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Hair Density/Melanoma'
normal_type_1 = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Skin Type/Normal/Type I'
mel_type_1 = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Skin Type/Melanoma/Type I'
normal_type_2 = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Skin Type/Normal/Type II'
mel_type_2 = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Skin Type/Melanoma/Type II'
normal_type_3 = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Skin Type/Normal/Type III'
mel_type_3 = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Skin Type/Melanoma/Type III'
normal_regions = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Anatomical Site/Normal'
mel_regions = f'../input/melanoma-h5-dataset/---Full Validation Sets---v{version}/---Full Validation Sets---/Anatomical Site/Melanoma'

In [ ]:
from sklearn.metrics import roc_auc_score
def get_acc(norm_path, mel_path, db_w=0.5, reg_w=0.5):
    norm_imgs = [os.path.join(norm_path, img) for img in os.listdir(norm_path)]
    mel_imgs = [os.path.join(mel_path, img) for img in os.listdir(mel_path)]
    correct_db = 0
    correct_norm = 0
    correct_ensemble = 0
    db_preds = []
    norm_preds = []
    ens_preds = []
    tru_preds = []
    for i, img in enumerate(norm_imgs):
        loaded = imgPrep.load_img(img, target_size=(256, 256))
        arr = imgPrep.img_to_array(loaded)/255.
        pred1 = tf.sigmoid(latent_net.predict(arr.reshape(1, 256, 256, 3)))
        pred2 = tf.sigmoid(model.predict(arr.reshape(1, 256, 256, 3)))
        correct_db += int(pred1 < 0.5)
        correct_norm += int(pred2 < 0.5)
        ens_pred = db_w*pred1+(reg_w*pred2)
        correct_ensemble += int(ens_pred < 0.5)
        norm_preds.append(np.array(pred2)[0][0]); db_preds.append(np.array(pred1)[0][0]);
        ens_preds.append(np.array(ens_pred)[0][0])
        tru_preds.append(0)
    for i, img in enumerate(mel_imgs):
        loaded = imgPrep.load_img(img, target_size=(256, 256))
        arr = imgPrep.img_to_array(loaded)/255.
        pred1 = tf.sigmoid(latent_net.predict(arr.reshape(1, 256, 256, 3)))
        pred2 = tf.sigmoid(model.predict(arr.reshape(1, 256, 256, 3)))
        correct_db += int(pred1 >= 0.5)
        correct_norm += int(pred2 >= 0.5)  
        ens_pred = db_w*pred1+(reg_w*pred2)
        correct_ensemble += int(ens_pred >= 0.5)
        norm_preds.append(np.array(pred2)[0][0]); db_preds.append(np.array(pred1)[0][0]);
        ens_preds.append(np.array(ens_pred)[0][0])
        tru_preds.append(1)
    norm_auc = roc_auc_score(tru_preds, norm_preds)
    db_auc = roc_auc_score(tru_preds, db_preds)
    ens_auc = roc_auc_score(tru_preds, ens_preds)
    total_imgs = (len(norm_imgs)+len(mel_imgs))
    print(f'LatentNet Accuracy: {correct_db/total_imgs}, AUC: {db_auc}')
    print(f'Standard Accuracy: {correct_norm/total_imgs}, AUC: {norm_auc}')
    print(f'Ensemble Accuracy: {correct_ensemble/total_imgs}, AUC: {ens_auc}')

In [ ]:
get_acc(normal_regions, mel_regions)

In [ ]:
get_acc(normal_hair, mel_hair)

In [ ]:
get_acc(normal_type_1, mel_type_1)

In [ ]:
get_acc(normal_type_2, mel_type_2)

In [ ]:
get_acc(normal_type_3, mel_type_3)